<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/fast_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os
import shutil
  
!git clone https://www.github.com/keras-team/keras-contrib.git \
  && cd keras-contrib \
  && pip install git+https://www.github.com/keras-team/keras-contrib.git \
  && python convert_to_tf_keras.py \
  && USE_TF_KERAS=1 python setup.py install

!pip install scipy==1.1.0

fatal: destination path 'keras-contrib' already exists and is not an empty directory.


In [3]:
import scipy
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
from keras.models import load_model, save_model

import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import numpy as np
import os
from glob import glob
from PIL import Image

import tensorflow as tf
from tensorflow.python.keras.backend import set_session, clear_session
tf.compat.v1.disable_v2_behavior()

from tqdm import tqdm

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [20]:
# source: https://www.machinecurve.com/index.php/2020/02/10/using-constant-padding-reflection-padding-and-replication-padding-with-keras/
from keras.layers import Layer

'''
  2D Reflection Padding
  Attributes:
    - padding: (padding_width, padding_height) tuple
'''
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

In [62]:
def conv_layer(x, num_filters, kernel_size, stride):
  reflection_size = kernel_size // 2
  reflection_padding = (reflection_size, reflection_size)
  x = ReflectionPadding2D(reflection_padding)(x)
  x = Conv2D(num_filters, kernel_size, stride)(x)
  return x

In [54]:
def residual_block(x, num_filters):
  input = x
  x = conv_layer(x, num_filters, 3, 1)
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)

  x = conv_layer(x, num_filters, 3, 1)
  x = InstanceNormalization()(x)

  x = Concatenate(axis=2)([input, x])
  return x

In [63]:
def deconv_layer(x, num_filters, kernel_size, stride, upsample_size):
  reflection_size = kernel_size // 2
  reflection_padding = (reflection_size, reflection_size)
  x = ReflectionPadding2D(reflection_padding)(x)
  x = UpSampling2D(size=upsample_size)(x)
  x = Conv2D(num_filters, kernel_size, stride)(x)
  return x

In [66]:
def style_layer(input):
  x = conv_layer(input, 32, 9, 1) #conv1
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 64, 3, 2) # conv2
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 128, 3, 2) # conv3
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = residual_block(x, 128) # res1
  x = residual_block(x, 128) # res2
  x = residual_block(x, 128) # res3
  x = residual_block(x, 128) # res4
  x = residual_block(x, 128) # res5
  x = deconv_layer(x, 64, 3, 1, 2) # deconv1
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = deconv_layer(x, 32, 3, 1, 2) # deconv2
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 3, 9, 1) # deconv3? apparently.
  return x

In [67]:
input = Input(shape=(128, 128, 3))
style_layer(input)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (9, 1))`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 2))`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 2))`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1))`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 1))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 1))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call t

<tf.Tensor 'conv2d_95/BiasAdd:0' shape=(?, 518, 18180, 3) dtype=float32>